<a href="https://colab.research.google.com/github/palominojulio/5_ways_2D_histograms/blob/master/20250507_0_46_%5B_ESFERA_%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ──────────────────────────────
# 1. MONTAJE Y LIBRERÍAS
# ──────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

!pip install -q nibabel dicom2nifti pynrrd pydicom

import os
import shutil
import numpy as np
import nibabel as nib
import dicom2nifti
import nrrd
import pydicom
import pytz
from datetime import datetime
from google.colab import files

# ──────────────────────────────
# 2. FUNCIONES
# ──────────────────────────────
def leer_coordenadas_desde_fcsv(fcsv_path):
    with open(fcsv_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    data_lines = [line for line in lines if not line.startswith("#")]
    if not data_lines:
        raise ValueError("❌ El archivo .fcsv no contiene puntos válidos.")
    parts = data_lines[0].strip().split(",")
    x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
    if [x, y, z] == [0.0, 0.0, 0.0]:
        raise ValueError("⚠️ El punto aún es [0,0,0]. ¿Olvidaste guardar el fiducial desde Slicer?")
    return np.array([x, y, z])

# ──────────────────────────────
# 3. SUBIDA DEL ARCHIVO FCSV (FIDUCIAL)
# ──────────────────────────────
print("📌 Selecciona el archivo `.fcsv` exportado desde 3D Slicer")
uploaded = files.upload()
fcsv_path = list(uploaded.keys())[0]
print("✅ Archivo cargado:", fcsv_path)

coords_ras = leer_coordenadas_desde_fcsv(fcsv_path)
print("✅ Coordenadas RAS extraídas del fiducial:", coords_ras)

# ──────────────────────────────
# 4. NOMBRE DE LA CARPETA DICOM Y CONFIGURACIÓN
# ──────────────────────────────
base_dir = "/content/drive/MyDrive/0.UIA.PACS"
folder_name = input("📝 Escribí el nombre de la carpeta DICOM en 0.UIA.PACS:\n").strip()
radius_mm = float(input("🔵 Radio de la esfera en mm:\n"))

zona = pytz.timezone('Europe/Madrid')
timestamp = datetime.now(zona).strftime("%Y%m%d_%H%M")
result_dir = os.path.join(base_dir, f"{folder_name}_procesado_{timestamp}")
os.makedirs(result_dir, exist_ok=True)

base_out = f"{folder_name}_E{int(radius_mm)}mm"
dicom_dir = os.path.join(base_dir, folder_name)

# ──────────────────────────────
# 5. CONVERSIÓN DICOM → NIFTI
# ──────────────────────────────
temp_nifti = "/content/temp_nifti"
os.makedirs(temp_nifti, exist_ok=True)

dicom2nifti.convert_directory(dicom_dir, temp_nifti, compression=True)
nifti_files = [f for f in os.listdir(temp_nifti) if f.endswith(".nii.gz")]
if not nifti_files:
    raise FileNotFoundError("❌ No se generó ningún archivo .nii.gz")

nifti_path = os.path.join(temp_nifti, nifti_files[0])
img = nib.load(nifti_path)
data = img.get_fdata()
affine = img.affine
spacing = img.header.get_zooms()

inv_affine = np.linalg.inv(affine)
voxel_coords = np.round(nib.affines.apply_affine(inv_affine, coords_ras)).astype(int)

# ──────────────────────────────
# 6. MÁSCARA ESFÉRICA
# ──────────────────────────────
radius_vox = np.round(radius_mm / np.array(spacing)).astype(int)
zz, yy, xx = np.ogrid[:data.shape[2], :data.shape[1], :data.shape[0]]
dist = ((xx - voxel_coords[0])**2 / radius_vox[0]**2 +
        (yy - voxel_coords[1])**2 / radius_vox[1]**2 +
        (zz - voxel_coords[2])**2 / radius_vox[2]**2)
mask = dist <= 1.0
mask = mask.transpose(2, 1, 0)
subvolume = np.zeros_like(data)
subvolume[mask] = data[mask]

# ──────────────────────────────
# 7. GUARDADO
# ──────────────────────────────
nifti_out = os.path.join(result_dir, base_out + "_subvolume.nii.gz")
original_out = os.path.join(result_dir, base_out + "_original.nii.gz")
seg_out = os.path.join(result_dir, base_out + "_segmentacion.seg.nrrd")
fcsv_out = os.path.join(result_dir, base_out + "_fiducial_aneurisma.fcsv")

shutil.copy(nifti_path, original_out)
nib.save(nib.Nifti1Image(subvolume, affine), nifti_out)

seg_data = (subvolume > 0).astype(np.uint8)
seg_header = {
    'type': 'uint8',
    'dimension': 3,
    'sizes': seg_data.shape,
    'space': 'left-posterior-superior',
    'space origin': affine[:3, 3].tolist(),
    'space directions': [affine[:3, i].tolist() for i in range(3)],
    'encoding': 'gzip',
    'Segment0_Color': '1 0 0',
    'Segment0_Name': 'Aneurisma',
    'Segment0_ID': 'aneurisma_seg',
    'Segment0_LabelValue': '1',
    'Segment0_Extent': f"0 {seg_data.shape[0]-1} 0 {seg_data.shape[1]-1} 0 {seg_data.shape[2]-1}",
    'Segment0_ReferenceImageExtentOffset': '0 0 0'
}
nrrd.write(seg_out, seg_data, seg_header)
shutil.copy(fcsv_path, fcsv_out)

# ──────────────────────────────
# 8. LIMPIEZA Y CIERRE
# ──────────────────────────────
shutil.rmtree(temp_nifti, ignore_errors=True)

print("\n✅✅ Procesamiento completado")
print(f"📂 Carpeta de salida: {result_dir}")
print("📦 Archivos para cargar en 3D Slicer:")
print(f" - {os.path.basename(original_out)}")
print(f" - {os.path.basename(nifti_out)}")
print(f" - {os.path.basename(seg_out)}")
print(f" - {os.path.basename(fcsv_out)}  ← punto original copiado desde Slicer")


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 27.4 MB/s eta 0:00:00
📌 Selecciona el archivo `.fcsv` exportado desde 3D Slicer


Saving New Fiducial.fcsv to New Fiducial.fcsv
✅ Archivo cargado: New Fiducial.fcsv
✅ Coordenadas RAS extraídas del fiducial: [  -5.41139178 -189.09345152 -497.6916529 ]
📝 Escribí el nombre de la carpeta DICOM en 0.UIA.PACS:
0.ATC.36519851216B182MB
🔵 Radio de la esfera en mm:
15



✅✅ Procesamiento completado
📂 Carpeta de salida: /content/drive/MyDrive/0.UIA.PACS/0.ATC.36519851216B182MB_procesado_20250507_0041
📦 Archivos para cargar en 3D Slicer:
 - 0.ATC.36519851216B182MB_E15mm_original.nii.gz
 - 0.ATC.36519851216B182MB_E15mm_subvolume.nii.gz
 - 0.ATC.36519851216B182MB_E15mm_segmentacion.seg.nrrd
 - 0.ATC.36519851216B182MB_E15mm_fiducial_aneurisma.fcsv  ← punto original copiado desde Slicer
